# Drug ontologies

lamindb provides access to the following public protein ontologies:

1.  [Drug Ontology](https://bioportal.bioontology.org/ontologies/DRON)

## Setup

In [ ]:
!lamin load test-ontologies

In [ ]:
import lnschema_bionty as lb
import pandas as pd

# adds an entry "human" into an empty instance
lb.settings.organism = "human"

## Bionty objects

Let us create a public knowledge accessor with {meth}`~lnschema_bionty.dev.BioRegistry.bionty`, which chooses a default public knowledge source from {meth}`~lnschema_bionty.BiontySource`. It's a [Bionty](https://lamin.ai/docs/bionty/bionty.bionty) object, which you can think about as a less-capable registry:

In [ ]:
drug_bt = lb.Drug.bionty()
drug_bt

As for registries, you can export the ontology as a `DataFrame`:

In [ ]:
df = drug_bt.df()
df.head()

Unlike registries, you can also export it as a Pronto object via `protein_bt.ontology`.

## Look up terms

As for registries, terms can be looked up with auto-complete:

In [ ]:
lookup = drug_bt.lookup()

The `.` accessor provides normalized terms (lower case, only contains alphanumeric characters and underscores):

In [ ]:
lookup.ac3

To look up the exact original strings, convert the lookup object to dict and use the `[]` accessor:

In [ ]:
lookup_dict = lookup.dict()
lookup_dict["AC3"]

By default, the `name` field is used to generate lookup keys. You can specify another field to look up:

In [ ]:
lookup = drug_bt.lookup(drug_bt.gene_symbol)

If multiple entries are matched, they are returned as a list:

In [ ]:
lookup.rab4a

## Search terms

Search behaves in the same way as it does for registries:

In [ ]:
drug_bt = lb.Protein.bionty()
drug_bt.search("RAB4A").head(3)

By default, search also covers synonyms:

In [ ]:
drug_bt.search("member of RAS oncogene family like 2B").head(3)

You can turn this off synonym by passing `synonyms_field=None`:

In [ ]:
drug_bt.search("RAS", synonyms_field=None).head(3)

Search another field (default is `.name`):

In [ ]:
drug_bt.search(
    "RABL2B", field=drug_bt.gene_symbol
).head()

## Standardize protein identifiers

Let us generate a `DataFrame` that stores a number of protein identifiers, some of which corrupted:

In [ ]:
df_orig = pd.DataFrame(
        index=[
            "A0A024QZ08",
            "X6RLV5",
            "X6RM24",
            "A0A024QZQ1",
            "This protein does not exist",
        ]
    )
df_orig

We can check whether any of our values are validated against the ontology reference:

In [ ]:
validated = drug_bt.validate(df_orig.index, drug_bt.uniprotkb_id)
df_orig.index[~validated]

## Ontology source versions

For any given entity, we can choose from a number of versions:

In [ ]:
lb.BiontySource.filter(entity="Protein").df()

When instantiating a Bionty object, we can choose a source or version:

In [ ]:
bionty_source = lb.BiontySource.filter(source="uniprot", version="2023-03", organism="human").one()
drug_bt = lb.Protein.bionty(bionty_source=bionty_source)
drug_bt

The currently used ontologies can be displayed using:

In [ ]:
lb.BiontySource.filter(currently_used=True).df()